<a href="https://colab.research.google.com/github/microprediction/automl-notebooks/blob/main/univariate_autosklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall numpy -y

In [ ]:
!pip install auto-sklearn
!pip install --upgrade pandas 

In [8]:
import pandas as pd 
import numpy as np 
from pprint import pprint
import sklearn.metrics
import autosklearn.regression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
from pprint import pprint

# Data to fit 

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/microprediction/schooled/main/data/sarima/sk_autoarima/train_1.csv',header=None)
last_col = df.columns[-1]
SCALE = 0.1
SEQ_LEN = 20
y = (df[last_col].values-X[:,-1])*SCALE
del df[last_col]
X = df.values[:,-SEQ_LEN:]*SCALE
X[:2,:4],y[:2], len(y), np.shape(X) 

(array([[ 0.0365374 ,  0.03573352,  0.04829583,  0.02249417],
        [ 0.0175058 , -0.01363301, -0.01982509,  0.02370967]]),
 array([ 0.0621914 , -0.06293862]),
 10000,
 (10000, 20))

In [21]:
import numpy as np
np.max(X), np.min(X), np.any(np.isnan(X)), np.max(y),np.min(y)

(0.5205376010401225,
 -0.4754179469018258,
 False,
 0.4771887771923248,
 -0.3983964988067798)

In [ ]:
X = X.astype(np.float32)
n_train = 5000
n_test = 4000
X_train, y_train = X[:n_train], y[:n_train]
X_test, y_test = X[n_train:(n_train+n_test)], y[n_train:(n_train+n_test)]
X_val, y_val = X[(n_train+n_test):], y[(n_train+n_test):]

automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder="./tmp",
)
automl.fit(X_train, y_train, dataset_name="diabetes")

In [23]:
print(automl.leaderboard())

          rank  ensemble_weight               type      cost  duration
model_id                                                              
15           1             0.72      liblinear_svr  0.024409  0.787402
11           2             0.06   gaussian_process  0.034652  2.157531
13           3             0.22  gradient_boosting  0.058039  1.923061


In [24]:
train_predictions = automl.predict(X_train)
print("Train R2 score:", sklearn.metrics.r2_score(y_train, train_predictions))
test_predictions = automl.predict(X_test)
print("Test R2 score:", sklearn.metrics.r2_score(y_test, test_predictions))
val_predictions = automl.predict(X_val)
print("Test R2 score:", sklearn.metrics.r2_score(y_val, val_predictions))
train_predictions = automl.predict(X_train)
print("Train MSE score:", sklearn.metrics.mean_squared_error(y_train, train_predictions))
test_predictions = automl.predict(X_test)
print("Test MSE score:", sklearn.metrics.mean_squared_error(y_test, test_predictions))
val_predictions = automl.predict(X_val)
print("Val MSE score:", sklearn.metrics.mean_squared_error(y_val, val_predictions))


print('Val MSE relative to last value:',sklearn.metrics.mean_squared_error(y_val, val_predictions)/sklearn.metrics.mean_squared_error(y_val, X_val[:,-1]))


Train R2 score: 0.9832637964122715
Test R2 score: 0.9618754229277182
Test R2 score: 0.9667491724264394
Train MSE score: 7.713655876562375e-05
Test MSE score: 0.00019537754469009904
Val MSE score: 0.00017328048512845487
Val MSE relative to last value: 0.6469804324263251
